In [1]:
import sys
#sys.path.append("/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection")
sys.path.append("/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2")
#sys.path.append("/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/training")
sys.argv = ["train.py"]

In [2]:
from training.detectors import DETECTOR

/opt/anaconda3/envs/lime/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/lime/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <FB2FD416-6C4D-3621-B677-61F07C02A3C5> /opt/anaconda3/envs/lime/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/lime/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/lime/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/lime/lib/python3.9/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/lime/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installe

## Imports

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from pytorch_grad_cam import XGradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from torchvision import transforms
from pytorch_grad_cam.utils.image import show_cam_on_image

### Load Xception Model

In [4]:
import yaml
from training.detectors import xception_detector
#i needed to append my analysis path to system to access the bcos saved models

def load_config(path, additional_args = {}):
    # parse options and load config
    with open(path, 'r') as f:
        config = yaml.safe_load(f)
    try:# KAI: added this, to ensure it finds the config file
        #with open('/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection/training/config/train_config.yaml', 'r') as f:
        with open('/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/training/config/train_config.yaml', 'r') as f:
            config2 = yaml.safe_load(f)
    except FileNotFoundError:
        #with open(os.path.expanduser('/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection/training/config/train_config.yaml'), 'r') as f:
        with open(os.path.expanduser('/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/training/config/train_config.yaml'), 'r') as f:
            config2 = yaml.safe_load(f)
    if 'label_dict' in config:
        config2['label_dict']=config['label_dict']
    config.update(config2)
    # config['local_rank']=args.local_rank
    if config['dry_run']:
        config['nEpochs'] = 0
        config['save_feat']=False
    for key, value in additional_args.items():
        config[key] = value
    return config

#path = "/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection/training/config/detector/xception.yaml"
path = "/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/training/config/detector/xception.yaml"
additional_args = {'test_batchSize': 12, 
                   'pretrained':
                    #f'/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection/weights/resnet/ckpt_best.pth'
                    f'/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/weights/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-10-17-17/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-12-19-31/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-17-23-41/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-21-26-03/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-07-21-43-33/test/avg/ckpt_best.pth'
                    #f'{weights_base_path}xception_2025-02-08-09-13-00/test/avg/ckpt_best.pth'
                  }
config = load_config(path, additional_args=additional_args)
print("Registered models:", DETECTOR.data.keys())
model_class = DETECTOR[config['model_name']]
model = model_class(config)

model.eval()

Registered models: dict_keys(['vgg19_bcos', 'resnet34', 'resnet34_bcos', 'resnet34_bcos_v2', 'inception_bcos_detector', 'xception'])


/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/training/detectors/xception_detector.py:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = 

XceptionDetector(
  (backbone): Xception(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (block1): Block(
      (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (rep): Sequential(
        (0): SeparableConv2d(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Re

### Loading in grid 

In [5]:
class MyToTensor(transforms.ToTensor):
    def __init__(self):
        """
        Same as transforms.ToTensor, except that if input to __call__ is already tensor, the input is returned unchanged
        """
        super().__init__()

    def __call__(self, input_img):
        if not isinstance(input_img, torch.Tensor):
            return super().__call__(input_img)
        return input_img
    
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize if necessary
    MyToTensor(),            # Converts image to tensor if not already          
])


class CustomImageDataset(Dataset):
    def __init__(self, folder_paths, transform=None):
        """
        Args:
            folder_paths (dict): Dictionary where keys are folder paths, and values are the labels.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.image_files = []  # Store (image_path, label) tuples
        for fp, label in folder_paths.items():
            # List all subdirectories within the folder
            subfolders = [os.path.join(fp, d) for d in os.listdir(fp) if os.path.isdir(os.path.join(fp, d))]
        # Include images from the root directory as well as subfolders
        self.image_files.extend(
            [(os.path.join(fp, f), label) for f in os.listdir(fp) if f.endswith((".png", ".jpg"))]
        )

        for folder_path in subfolders:
            self.image_files.extend(
                [(os.path.join(folder_path, f), label) for f in os.listdir(folder_path) if f.endswith((".png", ".jpg"))]
            )
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path, label = self.image_files[idx]  # Get image path and its label
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label, img_path


#file_path_deepfakebench = {"/Users/Linus/Desktop/GIThubXAIFDEEPFAKE/Interpretable-Deep-Fake-Detection/datasets/GPG_grids/3ch": 1}
file_path_deepfakebench = {"/Users/msrobin/GitHub Repositorys/Interpretable-Deep-Fake-Detection-2/datasets/2x2_images": 1}

dataset = CustomImageDataset(file_path_deepfakebench, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for images, label, img_path in dataloader:
    print(f"Batch of images shape: {images.shape}")
    break

Batch of images shape: torch.Size([20, 3, 224, 224])


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for img_batch, label_batch, path_batch in dataloader:
    print(f"Batch of images shape: {img_batch.shape}")

    # Iterate through the first 5 images in the batch
    num_images = min(len(img_batch), 20)
    for i in range(num_images):
        img = img_batch[i].unsqueeze(0).to(device)  # Process a single image
        label = label_batch[i]
        img_path = path_batch[i]
        img = img.requires_grad_(True)
        
        data_dict = {'image': img, 'label': label}
        
        model.zero_grad()
        out = model(data_dict)
        
        prob = out['prob']
        print('predicted prob', prob)


Batch of images shape: torch.Size([20, 3, 224, 224])
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
predicted prob tensor([0.5043], grad_fn=<SelectBackward0>)
pre

## X Grad Cam

In [7]:
import torch.nn as nn

In [8]:
#to find last layer automatically 
def find_last_conv_layer(model):
    """Finds the last convolutional layer in the model."""
    for layer in reversed(list(model.modules())):
        if isinstance(layer, nn.Conv2d):
            return layer
    return None  # Fallback in case no Conv2d is found


TARGET_LAYER = find_last_conv_layer(model.backbone)

# Check if a valid layer was found
if TARGET_LAYER is None:
    raise ValueError("No convolutional layer found in the model!")


In [9]:
TARGET_LAYER

Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1))

In [ ]:
cam = XGradCAM(model=model, target_layers=[TARGET_LAYER])

for img_batch, label_batch, path_batch in dataloader:
    print(f"Batch of images shape: {img_batch.shape}")

    # Iterate through the first 5 images in the batch
    num_images = min(len(img_batch), 20)
    for i in range(num_images):
        img = img_batch[i].unsqueeze(0).to(device)  # Process a single image
        label = label_batch[i]
        img_path = path_batch[i]
        print(img.shape)

        img_np = np.transpose(img[0].cpu().numpy(), (1, 2, 0))  # Convert to NumPy
        img = img.requires_grad_(True)

        # Prepare input for XGrad-CAM
        data_dict = {'image': img, 'label': label}

        model.zero_grad()
        out = model(data_dict)
        print(type(out['prob']))
        # Generate XGrad-CAM heatmap
        targets = [ClassifierOutputTarget(label.cpu().item())]  # Ensure correct format  
        # Make sure the image tensor has the correct shape
        if img.dim() == 3:
            img = img.unsqueeze(0)  # Convert (C, H, W) → (1, C, H, W)
        print(img.shape)
        # Run the model (returns a dictionary)
        out = model(data_dict)

        # Extract raw tensor from the dictionary
        out = out['cls']  # ✅ Extract class predictions

        # Ensure target is an integer
        targets = [ClassifierOutputTarget(out.argmax(dim=1).item())]  # ✅ Get class label
        print(targets)

        # Run Grad-CAM
        grayscale_cam = cam(input_tensor=data_dict['image'], targets=targets)[0]

        # Extract first image's CAM
        # After Grad-CAM, squeeze only the batch dim
        print(grayscale_cam)
        grayscale_cam = grayscale_cam.squeeze(0)  # Shape: (224, 224)


        # Overlay the heatmap on the image
        heatmap = cv2.applyColorMap(np.uint8(255 * grayscale_cam), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255  # Normalize for correct blending
        overlay = cv2.addWeighted(img_np, 0.6, heatmap, 0.4, 0)

        # Ensure LIME gets the correct input (original or overlay?)
        fake_pred, pixel_counts = lime_grid_eval(img_np)  # Use original image for LIME

        print(fake_pred, pixel_counts)

        # Plot the original image with the XGrad-CAM heatmap overlay
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        true_fake_pos = int(img_path.split('_fake_')[1].split('.')[0])

        # Original image
        ax[0].imshow(img_np)
        ax[0].axis('off')
        ax[0].set_title(f"Original Image: {true_fake_pos}")

        # Image with XGrad-CAM heatmap
        ax[1].imshow(overlay)
        ax[1].axis('off')
        ax[1].set_title(f'XGrad-CAM Prediction: {fake_pred}')

        plt.show()


Batch of images shape: torch.Size([20, 3, 224, 224])
torch.Size([1, 3, 224, 224])
<class 'torch.Tensor'>
torch.Size([1, 3, 224, 224])


AttributeError: 'str' object has no attribute 'shape'